In [ ]:
from functools import partial
import ipyvuetify as v
from sepal_ui import sepalwidgets as sw
from pathlib import Path

from scripts import run
from utils import utils
from utils import messages as ms
from utils import parameters as pm

In [ ]:
#use a class to define your input and output in order to have mutable variables
class Gfc_export_io:
    def __init__(self):
        #set up output
        self.clip_map = None
        self.proj_map = None
        
gfc_export_io = Gfc_export_io()

In [ ]:
#create the tile with the links
id_ = "gfc_export_widget"
title = "Results"
    
gfc_results = sw.Tile(id_, title, inputs=[''])

In [ ]:
#create the output alert 
gfc_export_output = sw.Alert()#.add_msg(ms.GFC_EXPORT)

#create the button that start your process
gfc_export_btn = sw.Btn('Export data')

#create a process tile
id_ = "gfc_export_widget"
title = 'Export the data'

gfc_export = sw.Tile(
    id_, 
    title, 
    btn=gfc_export_btn,  
    output=gfc_export_output,
    inputs=['']
)

#bind the button to the process by writing a custom function
def process_start(widget, event, data, output, gfc_results):
    
    asset = gfc_aoi_io.assetId
    threshold = gfc_viz_io.threshold
    visualization = gfc_viz_io.visualization
    
    #toggle the loading button
    widget.toggle_loading()
    
    #check inputs
    if not output.check_input(asset, ms.NO_AOI): return widget.toggle_loading()
    if not output.check_input(threshold, ms.NO_THRESHOLD): return widget.toggle_loading()
    if not output.check_input(visualization, ms.NO_VIZ): return widget.toggle_loading()
    
    try:
        #retreive the data
        clip_map, csv_file, result_layout = run.gfcExport(asset, threshold, output)
        
        #save the clip_map variable
        setattr(gfc_export_io, 'clip_map', clip_map)
        setattr(gfc_export_io, 'proj_map', pm.getGfcDir() + Path(clip_map).stem + '_proj.tif')
    
        gfc_results.set_content(result_layout)
        
    except Exception as e: 
        output.add_live_msg(str(e), 'error')
        
    #toggle the loading button
    widget.toggle_loading()
    
    return 

gfc_export_btn.on_event('click', partial(
    process_start, 
    output = gfc_export_output,
    gfc_results = gfc_results
))

In [ ]:
#this tiles will only be displayed if you launch voila from this file 
gfc_export

In [ ]:
#this tiles will only be displayed if you launch voila from this file 
gfc_results